# ESG Theme Classification Heatmap Demo

This notebook demonstrates how to classify ESG (Environmental, Social, Governance) themes in text and visualize the results as a heatmap.

# Required Libraries

This notebook requires the following libraries (install via requirements.txt):
- spacy (with en_core_web_sm)
- transformers
- pandas
- matplotlib
- seaborn

In [ ]:
# Input Document
# Paste your CSR (Corporate Social Responsibility) report or similar text document below

input_text = """Example CSR report text. 
This is a placeholder. Please replace this with your actual CSR report or text document.
"""

In [ ]:
# Sentence Splitting with spaCy

# Import the spaCy library
import spacy

# Load the English language model (small version)
nlp = spacy.load("en_core_web_sm")

# Process the input text with spaCy
# This creates a Doc object with linguistic annotations
doc = nlp(input_text)

# Extract sentences using spaCy's sentence segmentation
# spaCy identifies sentence boundaries based on punctuation and other linguistic features
sentences = [sent.text.strip() for sent in doc.sents]

# Print the list of sentences for verification
print(f"Found {len(sentences)} sentences:")
for i, sentence in enumerate(sentences, 1):
    print(f"{i}. {sentence}")